In [1]:
#comment_text длина кол-во слов, предложен, средняя длина предлож кол-во знаков препин кол-во уникальных слов, средняя длина слова
import pandas as pd
import pandas_profiling as pp
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import scipy.sparse
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC

In [2]:
data_train = pd.read_csv(r'E:\PycharmProjects\Machine_L\Data_Science_Club\Hackathon\train (1).csv')
data_test = pd.read_csv(r'E:\PycharmProjects\Machine_L\Data_Science_Club\Hackathon\test (1).csv')

c:\python36\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data_test.drop(['month_created'], axis = 1, inplace = True)

In [4]:
data_test.head(1)

,comment,comment_id,company_id,date_created,product_id,rating,user_agent,user_date_created,user_id,user_ip
0,"спасибо все отлично, качество как на картинке,...",1655822,2214161,2018-03-19 21:36:07,1301549.0,5,NaN,2017-11-01 00:00:46,12969242,NaN


In [5]:
target = data_train.is_fake

In [6]:
data_train= data_train[data_test.columns.tolist()]
data_train['is_fake'] = target

In [7]:
data_train.head()

,comment,comment_id,company_id,date_created,product_id,rating,user_agent,user_date_created,user_id,user_ip,is_fake
0,компания не выполнила заказ без обьяснения пр...,1,2169053,2014-10-23 20:26:44,6.0,1,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,2010-02-03 21:03:22,91933,178.92.70.220,False
1,"Впервые встретился такой професиональный, отве...",2,709306,2016-12-10 08:38:27,2.0,5,Mozilla/5.0 (Linux; Android 5.1.1; C6602 Build...,2010-02-03 21:03:22,91933,94.179.59.162,False
2,"Несерйозная, некоректная менеджер інтернет маг...",3,35287,2017-02-11 15:55:14,NaN,1,Mozilla/5.0 (Linux; Android 6.0; ASUS_X008D Bu...,2010-02-03 21:03:22,91933,37.73.137.188,False
3,"Не рекомендую. Не отвечают, не выполняют заказы.",4,1778322,2017-06-26 18:00:45,1.0,1,Mozilla/5.0 (Linux; Android 6.0; ASUS_X008D Bu...,2010-02-03 21:03:22,91933,37.73.149.180,False
4,"На письма не отвечают, заказы не выполняют. Не...",5,2821533,2017-07-06 16:57:40,NaN,1,Mozilla/5.0 (Linux; Android 6.0; ASUS_X008D Bu...,2010-02-03 21:03:22,91933,37.73.221.126,False


In [8]:
data_train.is_fake.value_counts()

False    1276801
True        4698
Name: is_fake, dtype: int64

In [9]:
data_train.loc[:, 'is_fake'] = data_train.loc[:, 'is_fake'].replace(to_replace = [False, True], value = [0, 1])

In [10]:
all_text = pd.concat([data_train['comment'], data_test['comment']])

In [11]:
fakes = data_train[data_train.is_fake == 1]
trues = data_train[data_train.is_fake == 0]
fakes.index = [i for i in range(fakes.shape[0])]
trues.index = [i for i in range(trues.shape[0])]

In [12]:
#working with fakes
fakes.comment[0]

'Огромное спасибо за посылку! Крылья бабочек-супер! Удачи вам!'

In [13]:
fakes.comment[0].replace('-', ' ')

'Огромное спасибо за посылку! Крылья бабочек супер! Удачи вам!'

In [14]:
lens_fakes_comments = [len(fakes.comment[i]) for i in range(fakes.shape[0])] #длины фейковых комментариев

In [15]:
lens_trues_comments = [len(trues.comment[i]) for i in range(trues.shape[0])]#длины настоящих комментариев

In [16]:
nb_words_fakes = [len(fakes.comment[i].replace('-', ' ').split()) for i in range(fakes.shape[0])] #кол-во слов в фейковых комментах

In [17]:
nb_words_trues = [len(trues.comment[i].replace('-', ' ').split()) for i in range(trues.shape[0])] #кол-во слов в настоящих комментах

In [18]:
nb_unique_words_fakes = [len(np.unique(fakes.comment[i].replace('-', ' ').split())) for i in range(fakes.shape[0])] #кол-во уникальных слов в фейковых комментах

In [19]:
nb_unique_words_trues = [len(np.unique(trues.comment[i].replace('-', ' ').split())) for i in range(trues.shape[0])] #кол-во уникальных слов в настоящих комментах

In [20]:
freq_words_fakes = list(np.array(nb_unique_words_fakes) / np.array(nb_words_fakes))

In [21]:
freq_words_trues = list(np.array(nb_unique_words_trues) / np.array(nb_words_trues))

c:\python36\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


In [22]:
min(freq_words_fakes)

0.5714285714285714

In [23]:
min(freq_words_trues)

0.029411764705882353

In [24]:
trues[trues.index == freq_words_trues.index(min(freq_words_trues))]

,comment,comment_id,company_id,date_created,product_id,rating,user_agent,user_date_created,user_id,user_ip,is_fake
290775,good good good good good good good good good g...,331429,988607,2014-11-04 08:19:11,NaN,5,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,2013-09-23 15:11:52,1359135,195.95.210.249,0


In [25]:
data_train.loc[:, 'lens_comments'] = 0

In [26]:
data_train.loc[:, 'nb_words'] = 0

In [27]:
data_train.loc[:, 'nb_unique_words'] = 0

In [28]:
data_train.loc[:, 'freq_words'] = 0

In [33]:
'''
data_train[data_train.is_fake == 1].loc[:, 'lens_comments'] = lens_fakes_comments
data_train[data_train.is_fake == 1].loc[:, 'nb_words'] = nb_words_fakes
data_train[data_train.is_fake == 0].loc[:, 'nb_words'] = nb_words_trues
data_train[data_train.is_fake == 1].loc[:, 'nb_unique_words'] = nb_unique_words_fakes
data_train[data_train.is_fake == 0].loc[:, 'nb_unique_words'] = nb_unique_words_trues
data_train[data_train.is_fake == 1].loc[:, 'freq_words'] = freq_words_fakes
data_train[data_train.is_fake == 0].loc[:, 'freq_words'] = freq_words_trues
'''

"\ndata_train[data_train.is_fake == 1].loc[:, 'lens_comments'] = lens_fakes_comments\ndata_train[data_train.is_fake == 1].loc[:, 'nb_words'] = nb_words_fakes\ndata_train[data_train.is_fake == 0].loc[:, 'nb_words'] = nb_words_trues\ndata_train[data_train.is_fake == 1].loc[:, 'nb_unique_words'] = nb_unique_words_fakes\ndata_train[data_train.is_fake == 0].loc[:, 'nb_unique_words'] = nb_unique_words_trues\ndata_train[data_train.is_fake == 1].loc[:, 'freq_words'] = freq_words_fakes\ndata_train[data_train.is_fake == 0].loc[:, 'freq_words'] = freq_words_trues\n"

In [32]:
data_train.loc[data_train.is_fake == 1, 'lens_comments'] = lens_fakes_comments

In [34]:
data_train.loc[data_train.is_fake == 0, 'lens_comments'] = lens_trues_comments

In [35]:
data_train.loc[data_train.is_fake == 1, 'nb_words'] = nb_words_fakes
data_train.loc[data_train.is_fake == 0, 'nb_words'] = nb_words_trues
data_train.loc[data_train.is_fake == 1, 'nb_unique_words'] = nb_unique_words_fakes
data_train.loc[data_train.is_fake == 0, 'nb_unique_words'] = nb_unique_words_trues
data_train.loc[data_train.is_fake == 1, 'freq_words'] = freq_words_fakes
data_train.loc[data_train.is_fake == 0, 'freq_words'] = freq_words_trues

In [36]:
data_train.head(2)

,comment,comment_id,company_id,date_created,product_id,rating,user_agent,user_date_created,user_id,user_ip,is_fake,lens_comments,nb_words,nb_unique_words,freq_words
0,компания не выполнила заказ без обьяснения пр...,1,2169053,2014-10-23 20:26:44,6.0,1,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,2010-02-03 21:03:22,91933,178.92.70.220,0,51,7,7,1.000000
1,"Впервые встретился такой професиональный, отве...",2,709306,2016-12-10 08:38:27,2.0,5,Mozilla/5.0 (Linux; Android 5.1.1; C6602 Build...,2010-02-03 21:03:22,91933,94.179.59.162,0,268,38,36,0.947368


In [42]:
data_train.loc[:, 'lens_comments':].to_csv('comment_train.csv', header = True, index = False)

In [46]:
lens_comments_test = [len(data_test.comment[i]) for i in range(data_test.shape[0])]

In [47]:
nb_words_test = [len(data_test.comment[i].replace('-', ' ').split()) for i in range(data_test.shape[0])]

In [48]:
nb_unique_words_test = [len(np.unique(data_test.comment[i].replace('-', ' ').split())) for i in range(data_test.shape[0])]

In [49]:
freq_words_test = list(np.array(nb_unique_words_test) / np.array(nb_words_test))

c:\python36\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


In [50]:
data_test['lens_comment'] = lens_comments_test
data_test['nb_words'] = nb_words_test
data_test['nb_unique_words'] = nb_unique_words_test
data_test['freq_words'] = freq_words_test

In [51]:
data_test.head()

,comment,comment_id,company_id,date_created,product_id,rating,user_agent,user_date_created,user_id,user_ip,lens_comment,nb_words,nb_unique_words,freq_words
0,"спасибо все отлично, качество как на картинке,...",1655822,2214161,2018-03-19 21:36:07,1301549.0,5,NaN,2017-11-01 00:00:46,12969242,NaN,71,9,9,1.000000
1,Заказывала шоколад Риттер Спорт 10 шт. К сожал...,1655823,2333068,2017-12-10 13:46:49,699700.0,5,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,2017-11-01 00:02:19,12969248,91.90.13.210,290,44,43,0.977273
2,"Быстро отзвонились, охотно отвечали на все воп...",1655824,752692,2017-11-01 00:07:38,NaN,5,Mozilla/5.0 (Windows NT 6.1; rv:38.0) Gecko/20...,2017-11-01 00:02:44,12969251,93.115.95.204,130,18,18,1.000000
3,Очень оперативный и внимательный к покупателю ...,1655825,334751,2017-11-20 23:40:01,1279247.0,5,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,2017-11-01 00:02:44,12969251,37.28.153.28,202,25,24,0.960000
4,Все прошло очень оперативно! Заказ привезли в ...,1655826,2263650,2017-11-20 23:39:51,1225545.0,5,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,2017-11-01 00:02:44,12969251,37.28.153.28,70,12,12,1.000000


In [52]:
data_test.loc[:, 'lens_comment':].to_csv('comment_test.csv', header = True, index = False)